In [1]:
import os
import sys
os.chdir('../')
SCRIPDIR = os.path.dirname(os.path.abspath("test.ipynb"))
sys.path.append(os.path.dirname(SCRIPDIR))
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def pick_a_sample(check_df: pd.DataFrame, result_df: pd.DataFrame = None, idx:int = None):
    if check_df[check_df['ERROR_REASON'].isna()].shape[0] == 0:
        print("No error found!")
        return None
    if idx is not None:
        sample = check_df.loc[[idx]]
    else:
        sample = check_df[check_df['ERROR_REASON'].isna() & check_df['IS_ERROR'] == True].sample()
    # if result_log is not None:
    print("-----------I have the schema generated as follows:---------")
        # logs = result_log.iloc[sample.LOGS_INDEX.values[0]]
    temp_df = result_df[result_df['TESTFILE_PATH'] == sample['TESTFILE_PATH'].values[0]]
    print(sample.index)
    for sql in temp_df[(temp_df['CASE_TYPE'] == 'Statement') | (temp_df.index == sample.index.values[0])]['SQL']:
        print(sql + ';')
        # print(logs.values[0])
    print("-----------The SQL commands is:---------")
    print(sample.SQL.values[0])
    print("-----------The expected result is:---------")
    print(sample.EXPECTED_RESULT.values[0])
    print("-----------The actual result is:---------")
    print(sample.ACTUAL_RESULT.values[0])
    print("-----------The error message is:---------")
    print(sample.ERROR_MSG.values[0])

    print("-----------The test file is:---------")
    print(sample.TESTFILE_PATH.values[0])
    print("___________The index is:___________")
    print(sample.index.values[0])
    return sample.index.values[0]

# SQLite

In [ ]:
dbms_name = 'sqlite'

output_path = f"output/{dbms_name}_duckdb_sample_100.csv"
output_reason_path = f"output/{dbms_name}_duckdb_sample_100_reason.csv"


In [ ]:
# open a csv file and read the content
df = pd.read_csv(f"output/{dbms_name}_duckdb_filter_results.csv")
result_log = pd.read_csv(f"output/{dbms_name}_duckdb_filter_logs.csv")
error_percentage = df[df['IS_ERROR'] == True].shape[0] / df.shape[0]
print(1 - error_percentage)
error_queries = df[(df['CASE_TYPE'] == 'Query') & (df['IS_ERROR'] == True)].shape[0]
error_statements = df[(df['CASE_TYPE'] == 'Statement')& (df['IS_ERROR'] == True)].shape[0]
wrong_queries = df[(df['CASE_TYPE'] == 'Query') & (df['ERROR_MSG'] == 'Result MisMatch')].shape[0]
print(f"Queries: {error_queries}, Statements: {error_statements}")
print(f"Wrong Queries: {wrong_queries}")

# add empty column ERROR_REASON to df
df['ERROR_REASON'] = None
check_df = df.copy()

In [ ]:
pick_a_sample(check_df, df)

In [ ]:
from src import utils
from copy import copy
import re

TEMP_FILTER = {
    'TYPE_CAST': lambda x: re.match(r'unrecognized token: \":\"', str(x['ERROR_MSG'])) is not None,
    'RANGE': lambda x: re.search(r'range\((.*?)\)', str(x['SQL']), re.DOTALL | re.IGNORECASE) is not None,
    "GENERAL": lambda x: re.search(r'no such function: .*', str(x['ERROR_MSG']), re.IGNORECASE) is not None,
    "EXPLAIN": lambda x: re.match(r'EXPLAIN', str(x['SQL'])) is not None,
    "SYNTAX": lambda x: re.search(r'syntax error', str(x['ERROR_MSG']), re.IGNORECASE) is not None,
}

new_df = copy(df)
reasons = pd.read_csv("data/duckdb_suite_errors.csv")
tags = reasons[reasons['DBMS'] == dbms_name]['TAG'].values.tolist()
for tag in tags:
    if tag not in TEMP_FILTER:
        print(f"Tag {tag} is not in the test filter, implement it first!")
        continue
    new_df.loc[new_df.apply(TEMP_FILTER[tag], axis=1) & new_df['IS_ERROR'] == True, 'ERROR_REASON'] = tag
    print(tag)
    print(reasons[reasons['TAG']==tag]['DESCRIPTION'].values[0])
    # print(new_df[new_df['ERROR_REASON']==tag].info())
    print(new_df[new_df['ERROR_REASON']==tag].shape[0])

check_df = copy(new_df[new_df['IS_ERROR'] == True & new_df['ERROR_REASON'].isna()])
print("Result MisMatch: ", check_df[check_df['ERROR_MSG'] == 'Result MisMatch'].shape[0])

In [ ]:
errors = new_df[new_df['IS_ERROR'] == True]
sample_100 = errors.sample(n=100, random_state=233)


In [ ]:
sample_100.to_csv(output_path, index=False)

In [ ]:
cnt=0

In [ ]:
sample = sample_100.iloc[cnt]

print("-----------I have the schema generated as follows:---------")
temp_df = new_df[new_df['TESTFILE_PATH'] == sample['TESTFILE_PATH']]
for sql in temp_df[temp_df['CASE_TYPE'] == 'Statement']['SQL']:
    print(sql + ';')
print("-----------The SQL commands is:---------")
print(sample.SQL)
print("-----------The expected result is:---------")
print(sample.EXPECTED_RESULT)
print("-----------The actual result is:---------")
print(sample.ACTUAL_RESULT)
print("-----------The error message is:---------")
print(sample.ERROR_MSG)
print(sample.LOGS_INDEX)
cnt += 1


In [ ]:
print(sample['TESTFILE_PATH'])

In [ ]:
sample_100_reason = pd.read_csv(output_path)
visible_reasons = sample_100_reason[[
    'SQL', 'ERROR_REASON', 'ERROR_MSG', 'ACTUAL_RESULT', 'EXPECTED_RESULT',]]
reasons_mapping = pd.read_csv('data/duckdb_suite_errors.csv')
reasons_mapping = reasons_mapping[reasons_mapping['DBMS'] == dbms_name]
# add columns to sample_100_reason according to reasons_mapping
visible_reasons = pd.merge(visible_reasons, reasons_mapping,
                           right_on='TAG', left_on='ERROR_REASON', how='inner')
# visible_reasons.drop(columns=['TAG'], inplace=True)
visible_reasons.to_csv(output_reason_path, index=False)


In [ ]:
reason_summary = visible_reasons[['SUPER_TAG',  'DESCRIPTION']].groupby(['SUPER_TAG']).count().style.to_latex()
print(reason_summary)

# PostgreSQL

In [3]:
dbms_name = 'postgresql'

output_path = f"output/{dbms_name}_duckdb_sample_100.csv"
output_reason_path = f"output/{dbms_name}_duckdb_sample_100_reason.csv"


In [4]:
# open a csv file and read the content
df = pd.read_csv(f"output/{dbms_name}_duckdb_filter_results.csv")
result_log = pd.read_csv(f"output/{dbms_name}_duckdb_filter_logs.csv")
error_percentage = df[df['IS_ERROR'] == True].shape[0] / df.shape[0]
print(1 - error_percentage)
error_queries = df[(df['CASE_TYPE'] == 'Query') & (df['IS_ERROR'] == True)].shape[0]
error_statements = df[(df['CASE_TYPE'] == 'Statement')& (df['IS_ERROR'] == True)].shape[0]
wrong_queries = df[(df['CASE_TYPE'] == 'Query') & (df['ERROR_MSG'] == 'Result MisMatch')].shape[0]
print(f"Queries: {error_queries}, Statements: {error_statements}")
print(f"Wrong Queries: {wrong_queries}")

# add empty column ERROR_REASON to df
df['ERROR_REASON'] = None
check_df = df.copy()

0.4934779492570118
Queries: 6411, Statements: 4190
Wrong Queries: 735


In [ ]:
pick_a_sample(check_df, df)

In [ ]:
from src import utils
from copy import copy
import re

TEMP_FILTER = {}

new_df = copy(df)
reasons = pd.read_csv("data/duckdb_suite_errors.csv")
tags = reasons[reasons['DBMS'] == dbms_name]['TAG'].values.tolist()
for tag in tags:
    if tag not in TEMP_FILTER:
        print(f"Tag {tag} is not in the test filter, implement it first!")
        continue
    new_df.loc[new_df.apply(TEMP_FILTER[tag], axis=1) & new_df['IS_ERROR'] == True, 'ERROR_REASON'] = tag
    print(tag)
    print(reasons[reasons['TAG']==tag]['DESCRIPTION'].values[0])
    # print(new_df[new_df['ERROR_REASON']==tag].info())
    print(new_df[new_df['ERROR_REASON']==tag].shape[0])

check_df = copy(new_df[new_df['IS_ERROR'] == True & new_df['ERROR_REASON'].isna()])
print("Result MisMatch: ", check_df[check_df['ERROR_MSG'] == 'Result MisMatch'].shape[0])

In [ ]:
errors = new_df[new_df['IS_ERROR'] == True]
sample_100 = errors.sample(n=100, random_state=233)
output_path = f"output/{dbms_name}_duckdb_sample_100.csv"
output_reason_path = f"output/{dbms_name}_duckdb_sample_100_reason.csv"


In [ ]:
sample_100.to_csv(output_path, index=False)

In [ ]:
cnt=0

In [ ]:
sample = sample_100.iloc[cnt]

print("-----------I have the schema generated as follows:---------")
temp_df = new_df[new_df['TESTFILE_PATH'] == sample['TESTFILE_PATH']]
for sql in temp_df[temp_df['CASE_TYPE'] == 'Statement']['SQL']:
    print(sql + ';')
print("-----------The SQL commands is:---------")
print(sample.SQL)
print("-----------The expected result is:---------")
print(sample.EXPECTED_RESULT)
print("-----------The actual result is:---------")
print(sample.ACTUAL_RESULT)
print("-----------The error message is:---------")
print(sample.ERROR_MSG)
print(sample.LOGS_INDEX)
cnt += 1


In [ ]:
print(sample['TESTFILE_PATH'])

In [ ]:
sample_100_reason = pd.read_csv(output_path)
visible_reasons = sample_100_reason[[
    'SQL', 'ERROR_REASON', 'ERROR_MSG', 'ACTUAL_RESULT', 'EXPECTED_RESULT',]]
reasons_mapping = pd.read_csv('data/duckdb_suite_errors.csv')
reasons_mapping = reasons_mapping[reasons_mapping['DBMS'] == dbms_name]
# add columns to sample_100_reason according to reasons_mapping
visible_reasons = pd.merge(visible_reasons, reasons_mapping,
                           right_on='TAG', left_on='ERROR_REASON', how='inner')
# visible_reasons.drop(columns=['TAG'], inplace=True)
visible_reasons.to_csv(output_reason_path, index=False)


In [ ]:
reason_summary = visible_reasons[['SUPER_TAG',  'DESCRIPTION']].groupby(['SUPER_TAG']).count().style.to_latex()
print(reason_summary)

# MySQL

In [5]:
dbms_name = 'mysql'

output_path = f"output/{dbms_name}_duckdb_sample_100.csv"
output_reason_path = f"output/{dbms_name}_duckdb_sample_100_reason.csv"


In [6]:
# open a csv file and read the content
df = pd.read_csv(f"output/{dbms_name}_duckdb_filter_results.csv")
result_log = pd.read_csv(f"output/{dbms_name}_duckdb_filter_logs.csv")
error_percentage = df[df['IS_ERROR'] == True].shape[0] / df.shape[0]
print(1 - error_percentage)
error_queries = df[(df['CASE_TYPE'] == 'Query') &
                   (df['IS_ERROR'] == True)].shape[0]
error_statements = df[(df['CASE_TYPE'] == 'Statement') &
                      (df['IS_ERROR'] == True)].shape[0]
wrong_queries = df[(df['CASE_TYPE'] == 'Query') & (
    df['ERROR_MSG'] == 'Result MisMatch')].shape[0]
print(f"Queries: {error_queries}, Statements: {error_statements}")
print(f"Wrong Queries: {wrong_queries}")

# add empty column ERROR_REASON to df
df['ERROR_REASON'] = None
check_df = df.copy()


0.346919157935211
Queries: 7818, Statements: 5770
Wrong Queries: 358


In [ ]:
pick_a_sample(check_df, df)

In [ ]:
from src import utils
from copy import copy
import re

TEMP_FILTER = {
    'TYPE_CAST': lambda x: re.match(r'1064 \(42000\).*\'::', str(x['ERROR_MSG'])) is not None,
    'GENERAL': lambda x: re.match(r'1305 \(42000\)', str(x['ERROR_MSG'])) is not None,
}

new_df = copy(df)
reasons = pd.read_csv("data/duckdb_suite_errors.csv")
tags = reasons[reasons['DBMS'] == dbms_name]['TAG'].values.tolist()
for tag in tags:
    if tag not in TEMP_FILTER:
        print(f"Tag {tag} is not in the test filter, implement it first!")
        continue
    new_df.loc[new_df.apply(TEMP_FILTER[tag], axis=1) & new_df['IS_ERROR'] == True, 'ERROR_REASON'] = tag
    print(tag)
    print(reasons[reasons['TAG']==tag]['DESCRIPTION'].values[0])
    # print(new_df[new_df['ERROR_REASON']==tag].info())
    print(new_df[new_df['ERROR_REASON']==tag].shape[0])

check_df = copy(new_df[new_df['IS_ERROR'] == True & new_df['ERROR_REASON'].isna()])
print("Result MisMatch: ", check_df[check_df['ERROR_MSG'] == 'Result MisMatch'].shape[0])

In [ ]:
errors = new_df[new_df['IS_ERROR'] == True]
sample_100 = errors.sample(n=100, random_state=233)
output_path = f"output/{dbms_name}_duckdb_sample_100.csv"
output_reason_path = f"output/{dbms_name}_duckdb_sample_100_reason.csv"


In [ ]:
sample_100.to_csv(output_path, index=False)

In [ ]:
cnt=0

In [ ]:
sample = sample_100.iloc[cnt]

print("-----------I have the schema generated as follows:---------")
temp_df = new_df[new_df['TESTFILE_PATH'] == sample['TESTFILE_PATH']]
for sql in temp_df[temp_df['CASE_TYPE'] == 'Statement']['SQL']:
    print(sql + ';')
print("-----------The SQL commands is:---------")
print(sample.SQL)
print("-----------The expected result is:---------")
print(sample.EXPECTED_RESULT)
print("-----------The actual result is:---------")
print(sample.ACTUAL_RESULT)
print("-----------The error message is:---------")
print(sample.ERROR_MSG)
print(sample.LOGS_INDEX)
cnt += 1


In [ ]:
print(sample['TESTFILE_PATH'])

In [ ]:
sample_100_reason = pd.read_csv(output_path)
visible_reasons = sample_100_reason[[
    'SQL', 'ERROR_REASON', 'ERROR_MSG', 'ACTUAL_RESULT', 'EXPECTED_RESULT',]]
reasons_mapping = pd.read_csv('data/duckdb_suite_errors.csv')
reasons_mapping = reasons_mapping[reasons_mapping['DBMS'] == dbms_name]
# add columns to sample_100_reason according to reasons_mapping
visible_reasons = pd.merge(visible_reasons, reasons_mapping,
                           right_on='TAG', left_on='ERROR_REASON', how='inner')
# visible_reasons.drop(columns=['TAG'], inplace=True)
visible_reasons.to_csv(output_reason_path, index=False)


In [ ]:
reason_summary = visible_reasons[['SUPER_TAG',  'DESCRIPTION']].groupby(['SUPER_TAG']).count().style.to_latex()
print(reason_summary)